### Set Environment

In [36]:
import json
import pandas as pd

### List all Output paths

In [37]:
from glob2 import glob
jsonFiles = glob('/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/*/posts/*.json') #Can be used absolute or relative paths

### Merge all data:

In [50]:
tiktok_data = pd.DataFrame([])

def merge_JsonFiles(filename):
    for f1 in filename:
        global tiktok_data
        with open(f1, 'r') as infile:
            data = json.load(infile)
        tiktok_data = tiktok_data.append(pd.json_normalize(data), ignore_index = True)
        tiktok_data.to_csv("/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/tiktok.csv", index=False)

merge_JsonFiles(jsonFiles)

/var/folders/cq/1ccdbd_j1256m9m9sgf4q0t00000gp/T/ipykernel_19586/3402362571.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tiktok_data = tiktok_data.append(pd.json_normalize(data), ignore_index = True)
/var/folders/cq/1ccdbd_j1256m9m9sgf4q0t00000gp/T/ipykernel_19586/3402362571.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tiktok_data = tiktok_data.append(pd.json_normalize(data), ignore_index = True)
/var/folders/cq/1ccdbd_j1256m9m9sgf4q0t00000gp/T/ipykernel_19586/3402362571.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tiktok_data = tiktok_data.append(pd.json_normalize(data), ignore_index = True)
/var/folders/cq/1ccdbd_j1256m9m9sgf4q0t00000gp/T/ipykernel_19586/3402362571.py:8: FutureWarning: The frame.appen

### Save as .csv

AttributeError: 'list' object has no attribute 'to_csv'